In [4]:
################## IMPORT LIBRARIES ##################

from IPython.display import Audio
import numpy as np
import pandas as pd
from os.path import join as pjoin
import scipy.signal as sig
from IPython.display import Audio
from masp import shoebox_room_sim as srs
from scipy.io import wavfile
import sys

In [5]:
################## IMPORT MY MODULES ##################
sys.path.append('../src')

import helpers as hlp
import importlib
importlib.reload(hlp);

In [ ]:
# parameters: 
N_irs=5 # number of rirs to generate
fs_rir = 48000
fs_target = 48000 
maxlim = 1.8 # Maximum reverberation time
band_centerfreqs=np.array([16000]) #change this for multiband
writepath='~/joanna/my-use-examples-masp/generated-rirs'
# Mic characteristics
mic_specs = np.array([[0, 0, 0, 1]]) # omni-directional
# mic_specs = np.array([[1, 0, 0, 0.5]])    # cardioid looking to the front
# mic_specs = np.array([[-1, 0, 0, 0.25]])  # hypercardioid looking backwards

In [ ]:
# ---------------- CREATE METADATA FOR THE RANDOM ROOMS ---------------

df_rooms=pd.DataFrame()
# lists of random room dimensions:
df_rooms["room_x"] = list(np.random.uniform(low = 3., high = 30., size=N_irs))
df_rooms["room_y"] = list(df_rooms["room_x"] * np.random.uniform(low=0.5, high=1, size=N_irs)) #avoid tunnels
df_rooms["room_z"] = np.random.uniform(low = 2.5, high = 5., size=N_irs)
df_rooms["volume"] = df_rooms["room_x"] * df_rooms["room_y"] * df_rooms["room_z"]
df_rooms=df_rooms.sort_values(by=['volume'])

# list of random t60 values and matching it with room dimensions
# so that the biggest rooms get the highest rt60 values 
df_rooms["rt60_set"]  = np.sort(np.random.uniform(low = .1, high = 1, size=N_irs))
# add a variation to rt60 values:
df_rooms["rt60_set"] += df_rooms["rt60_set"]*np.random.uniform(low = -0.1, high = 0.1, size=N_irs)
df_rooms=df_rooms.reset_index(drop=True)

# list of random mono receiver position inside the room (& making sure its somewhere in the middle):
mic_pos_x = []
mic_pos_y = []
mic_pos_z = []
for k in range(N_irs):
    mic_pos_x.append(np.random.uniform(low = 0.35*df_rooms.loc[k,"room_x"], high = 0.65*df_rooms.loc[k,"room_x"]))
    mic_pos_y.append(np.random.uniform(low = 0.35*df_rooms.loc[k,"room_y"], high = 0.65*df_rooms.loc[k,"room_y"]))
    mic_pos_z.append(np.random.uniform(low = 1., high = 2.))

df_rooms["mic_pos_x"]=mic_pos_x
df_rooms["mic_pos_y"]=mic_pos_y
df_rooms["mic_pos_z"]=mic_pos_z

# source position always the same in reference to mic (close-mic):
for k in range(N_irs):
    src_pos = hlp.place_on_circle(np.array([df_rooms.loc[k,"mic_pos_x"],df_rooms.loc[k,"mic_pos_y"],df_rooms.loc[k,"mic_pos_z"]]),0.1,0)
    src_pos=src_pos[0]
    df_rooms.loc[k,"src_pos_x"]=src_pos[0]
    df_rooms.loc[k,"src_pos_y"]=src_pos[1]
    df_rooms.loc[k,"src_pos_z"]=src_pos[2]

# Add empty columns to be filled by a multiprocessing function
df_rooms['rt60_true'] = ''
df_rooms['rt60_masp_stats'] = ''
df_rooms['cd_masp_stats'] = ''
df_rooms['mfp_masp_stats'] = ''
df_rooms['rt30_meas'] = ''
df_rooms['rt20_meas'] = ''
df_rooms['edt_meas'] = ''
df_rooms['c50_meas'] = ''

# save randomly generated room characteristics:
# df_rooms.to_csv("room_info.csv")
df_rooms.head(5)


In [ ]:
# ---------------- GENERATE RIRS WITH MASP BASED ON  METADATA ---------------

from multiprocessing import Pool,Manager

def render_mono_ir(df_rooms,i):
    # Multiprocessing function which, based on the room info,
    # computes room simulation, renders and stores a RIR, and 
    # calculates several acoustic parameters (either based on 
    # room or based on the synthetic RIR)
    # Inputs:
    # df_rooms - input dataframe with room information 
    # i - index for a specific room
    # -------------------------------------------------------
    # pick 1 raw from the data frame
    a = df_rooms.iloc[i]
    # receiver position (mono mic): 
    rec = np.array([[a.mic_pos_x, a.mic_pos_y, a.mic_pos_z]])
    # source position
    src = np.array([[a.src_pos_x, a.src_pos_y, a.src_pos_z]])
    # room dimensions:
    room = np.array([a.room_x, a.room_y, a.room_z])
    # reverberation:
    rt60 = np.array([a.rt60_set])
    # Compute absorption coefficients for desired rt60 and room dimensions
    abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60)
    # Small correction for sound absorption coefficients:
    if sum(rt60_true-rt60>0.05*rt60_true)>0 :
        abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60_true + abs(rt60-rt60_true))
    # Generally, we simulate up to RT60:
    limits = np.minimum(rt60, maxlim)
    # Compute echogram:
    abs_echogram= srs.compute_echograms_mic(room, src, rec, abs_walls, limits, mic_specs)
    # Compute stats based on the room information: 
    rt60_stats,d_critical,d_mfpath= srs.room_stats(room, abs_walls, verbose=True)
    # Render RIR: 
    mic_rir = srs.render_rirs_mic(abs_echogram, band_centerfreqs, fs_rir)
    # Save RIR: 
    round_x=int(100*np.round(a.room_x,2))
    round_y=int(100*np.round(a.room_y,2))
    round_z=int(100*np.round(a.room_z,2))
    round_rt=int(100*np.round(rt60_true[0],2))
    filename=f"monoRIR_x{round_x}y{round_y}z{round_z}_rtms{round_rt}.wav"
    ir_file_path=pjoin(writepath, filename)
    wavfile.write(ir_file_path, fs_target, np.squeeze(mic_rir).astype(np.float32))
    # Compute stats based on the RIR: 
    bands=np.array([16, 31.5, 63, 125, 250, 500, 1000, 2000, 4000, 8000, 16000])
    rt30_meas,rt20_meas, edt_meas,c50_meas=hlp.compute_ir_stats(pjoin(writepath, filename),bands)  
    # Fill df_rooms with computed stats:
    df_rooms.loc[i,"ir_file_path"] = ir_file_path
    df_rooms.loc[i,"rt60_true"] = rt60_true[0]
    df_rooms.loc[i,"rt60_masp_stats"] = rt60_stats[0]
    df_rooms.loc[i,"cd_masp_stats"] = d_critical[0]
    df_rooms.loc[i,"mfp_masp_stats"] = d_mfpath
    df_rooms.loc[i,"rt30_meas"] =rt30_meas
    df_rooms.loc[i,"rt20_meas"] =rt20_meas
    df_rooms.loc[i,"edt_meas"] =edt_meas
    df_rooms.loc[i,"c50_meas"] =c50_meas
    # Plots for debugging:
    # plt.figure()
    # plt.plot(np.squeeze(mic_rir))
    # hlp.plot_scene_raw(room,rec,src,perspective="xy")
    return df_rooms.loc[i]


if __name__ == "__main__":
    with Pool(processes=8) as pool:
        # run render_mono_ir in parallel
        result = pool.starmap(render_mono_ir, [(df_rooms, idx) for idx in range(N_irs)])

df_rooms_with_stats = pd.concat(result, axis=1).T
# store the information abou the dataset:
# (rir file paths and all corresponding room and rir stats)
df_rooms_with_stats.to_csv(pjoin(writepath,"rir_info.csv"))
df_rooms_with_stats.head(5)


In [ ]:
# ----- LISTEN TO REVERBERANT AUDIOS: -----
for i in range(5):
    # room impulse response
    fs_rir, rir_data = wavfile.read(df_rooms_with_stats.loc[i,"ir_file_path"]) 
    # speech
    fs_speech, speech_data = wavfile.read("sounds/dial1p1.wav") 
    # create reverberant version of speech 
    revspeech_data=sig.fftconvolve(speech_data,rir_data, 'full', 0)
    # print corresponding rt60
    print("t60="+ str(df_rooms_with_stats.loc[i,"rt60_set"]))
    # audio player
    display(Audio(data=revspeech_data,rate=fs_speech))
